# Visualizing the Feature Maps

sources: [here](https://www.kaggle.com/code/magokecol/pytorch-feature-maps-visualizer-snake-version) and [here](https://towardsdatascience.com/how-to-visualize-convolutional-features-in-40-lines-of-code-70b7d87b0030)

In [11]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
from torchvision import models, transforms
import cv2
import matplotlib.pyplot as plt
import zipfile
import PIL

In [4]:
model = models.vgg19_bn(pretrained=True)
model.eval()
model = model.double()

/home/mila/s/sara.ebrahim-elkafrawy/.conda/envs/ffcv2/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/mila/s/sara.ebrahim-elkafrawy/.conda/envs/ffcv2/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG19_BN_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_BN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19_bn-c79401a0.pth" to /home/mila/s/sara.ebrahim-elkafrawy/.cache/torch/hub/checkpoints/vgg19_bn-c79401a0.pth
100%|██████████| 548M/548M [00:05<00:00, 100MB/s]  


In [35]:
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

In [9]:
# list cnn layers
layers = [layer for layer in model.children()]
print('Layers: {}'.format(len(layers)))
print('Layers[0]: {}'.format(len(layers[0])))
# for l in layers:
#     print(l)

Layers: 3
Layers[0]: 53


In [8]:
# global variable to work with GPU if possible
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available(): print('Thanks Mila!')
print(device)

Thanks Mila!
cuda:0


# Which pixels activated that feature map?

The idea is the following: we start with a picture containing random pixels. We apply the network in evaluation mode to this random image, calculate the average activation of a certain feature map in a certain layer from which we then compute the gradients with respect to the input image pixel values. Knowing the gradients for the pixel values we then proceed to update the pixel values in a way that maximizes the average activation of the chosen feature map.

I know that this might sound confusing so let’s explain it again in different words: The network weights are fixed, the network will not be trained, and we try to find an image that maximizes the average activation of a certain feature map by performing gradient descent optimization on the pixel values.

So what is this good for? Let’s say we are interested in the feature maps of layer i. We register a forward hook on layer i that, once the forward method of layer i is called, saves the features of layer i in a variable.

## This class serves to hook cnn layers

In [10]:
class SaveFeatures():
    def __init__(self, module, device=None):
        # we are going to hook some model's layer (module here)
        self.hook = module.register_forward_hook(self.hook_fn)
        self.device = device

    def hook_fn(self, module, input, output):
        # when the module.forward() is executed, here we intercept its
        # input and output. We are interested in the module's output.
        self.features = output.clone()
        if self.device is not None:
            self.features = self.features.to(device)
        self.features.requires_grad_(True)

    def close(self):
        # we must call this method to free memory resources
        self.hook.remove()

How would you use a SaveFeatures object?

Register your hook for layer `i` with activations = `SaveFeatures(list(self.model.children())[i])` and after you applied your model to the image with `model(img_var)` you can access the features the hook saved for us in activations.features. Remember to call the method close to free up used memory.

In [30]:
class FeatureMapVisualizer():
    def __init__(self, cnn, device, channels=3, layers_base=None, norm=None, denorm=None, save=None):
        self.model = cnn

        if layers_base is None:
            self.layers = self.model
        else:
            self.layers = layers_base
        
        self.channels = channels
        self.device = device

        mean = torch.tensor([0.485, 0.456, 0.406], dtype=torch.float32)
        std = torch.tensor([0.229, 0.224, 0.225], dtype=torch.float32)
            
        self.norm = norm
        self.denorm = denorm

        if norm is None:
            self.norm = transforms.Normalize(mean=mean.tolist(), std=std.tolist())

        if denorm is None:
            self.denorm = transforms.Normalize((-mean / std).tolist(), (1.0 / std).tolist())
            
        self.save = save

    def set_layers_base(self, layers):
        # sometime we want to access to layers in deeper levels
        # so we could call something like:
        # featureMap.set_layers_base([module for module in model.children()][5][1])
        self.layers = layers
        
    def optimize_img(self, activations, filter, img, learning_rate, opt_steps, verbose):
        
        size = img.shape[1]
        img = torch.from_numpy(img.astype(np.float32).transpose(2,0,1))
        
        img = self.norm(img).double()
        img_input = img.clone().detach().reshape(1, self.channels, size, size).to(self.device).requires_grad_(True)
        optimizer = torch.optim.Adam([img_input], lr=learning_rate, weight_decay=1e-6)

        for n in range(opt_steps):
            optimizer.zero_grad()
            self.output = self.model(img_input)
            # TODO: the idea is to find an input image that
            #       'illuminate' ONLY ONE feature map (filter here)
            # TODO: 1 test a loss function that punish current
            #       activation filter with the rest of the
            #       filters mean values in the layer
            # TODO: 2 test a loss function that punish current activation
            #       filter with all the rest of the filters mean value
            #       of more layers (all?)
            loss = -1 * activations.features[0, filter].mean()
            loss.backward()
            if verbose > 1:
                print('.', end='')
            #print(loss.clone().detach().cpu().item())
            optimizer.step()
        if verbose > 1:
            print()
        img = self.denorm(img_input.clone().detach()[0].type(torch.float32))
        img = img.cpu().numpy().transpose(1,2,0)
        return img
        

    def visualize(self, layer, filter, size=56, upscaling_steps=12, upscaling_factor=1.2, lr=0.1, opt_steps=20, blur=None, verbose=2):
        training = self.model.training
        self.model.eval()
        self.model = self.model.double().to(self.device)
        
        # 1. generate random image
        img = np.uint8(np.random.uniform(100, 160, (size, size, self.channels)))/255
        
        # 2. register hook
        activations = SaveFeatures(self.layers[layer], self.device)
        if verbose > 0:
            print('Processing filter {}...'.format(filter))

        for i in range(upscaling_steps): 
            if verbose > 1:
                print('{:3d} x {:3d}'.format(size,size), end='')

            img = self.optimize_img(activations, filter, img, learning_rate=lr, opt_steps=opt_steps, verbose=verbose)

            if i < upscaling_steps-1:
                size = int(size*upscaling_factor)
                # scale the image up upscaling_steps times
                img = cv2.resize(img, (size, size), interpolation = cv2.INTER_CUBIC)
                # blur image to reduce high frequency patterns
                if blur is not None: img = cv2.blur(img,(blur,blur))
            img = np.clip(img, 0, 1)

        if verbose > 0:
            print('preparing image...')
        activations.close()
        self.model.train(training)
        if self.save != None:
            self.save("layer_{:02d}_filter_{:03d}.jpg".format(layer, filter), img)
        return img
    
    # We return the mean of every activation value, but this could
    # be other metric based on convolutional output values.
    def get_activations(self, monitor, input, mean=True):

        training = self.model.training
        self.model.eval()
        self.model = self.model.double().to(self.device)

        activations = {}
        mean_acts = {}

        print('hooking layers {}'.format(monitor))
        for layer in monitor:
            activations[layer] = SaveFeatures(self.layers[layer], device=self.device)

        self.output = self.model(input.to(self.device))

        for layer in activations.keys():
            filters = activations[layer].features.size()[1]
            mean_acts[layer] = [activations[layer].features[0,i].mean().item() for i in range(filters)]

        print('unhooking layers.')        
        for layer in activations.keys():
            activations[layer].close()
            
        self.model.train(training)
        
        if mean:
            return mean_acts
        
        return activations

In [31]:
# We will save all generated images in this directory
images_dir = './images/'
# create images directory
if not os.path.exists(images_dir):
    os.makedirs(images_dir)

In [32]:
# We save images 
def save(name, img):
    global image_dir
    plt.imsave(images_dir + name, img)

## Define some variables

In [33]:
# we save in the variable 'monitor' every ReLU layers that appears
# after every convolutional layer (they present non negative data)
#monitor = [2,5,9,12,16,19,22,25,29,32,35,38,42,45,48,51]
#monitor = [i for i, layer in enumerate(layers[0]) if isinstance(layer, torch.nn.Conv2d)]
monitor = [i for i, layer in enumerate(layers[0]) if isinstance(layer, torch.nn.ReLU)]

# define mean and std used for most famous images datasets
mean = torch.tensor([0.485, 0.456, 0.406], dtype=torch.float32)
std = torch.tensor([0.229, 0.224, 0.225], dtype=torch.float32)

# define global transformations based on previous mean and std
normalize = transforms.Normalize(mean=mean.tolist(), std=std.tolist())
denormalize = transforms.Normalize((-mean / std).tolist(), (1.0 / std).tolist())

# The input images will be prepared with this transformation
# Minimum image size recommended for input is 224
img2tensor = transforms.Compose([transforms.Resize((224,224)), transforms.ToTensor(), normalize])

upscaling_steps=13
opt_steps=20
verbose=1
TOP = 4
    

In [34]:
# Lets generate feature map visualization for filter 91 at layer 48
# to discover the best (or a good one) 'texture image' that activate this filter
fmv = FeatureMapVisualizer(model, device, layers_base=layers[0], save=None)

steps = 15
if str(device) == 'cpu': steps = 5
    
feature_map_48_91_img = fmv.visualize(
                            layer=48, filter=91, size=56,
                            upscaling_steps=steps, opt_steps=20, blur=3, verbose=2
                        )
plt.imsave(images_dir + 'input_for_L48F091.jpg', feature_map_48_91_img)

Processing filter 91...
 56 x  56....................
 67 x  67....................
 80 x  80....................
 96 x  96....................
115 x 115....................
138 x 138....................
165 x 165....................
198 x 198....................
237 x 237....................
284 x 284....................
340 x 340....................
408 x 408....................
489 x 489....................
586 x 586....................
703 x 703....................
preparing image...


Great, we can now access the feature maps of layer `i`! The feature maps could i.e. have the shape `[1, 512, 7, 7]` where `1` is the batch dimension, `512` the number of filters/feature maps and `7` the height and width of the feature maps. **The goal is to maximize the average activation of a chosen feature map `j`.** 

We, therefore, define the following loss function: 
    `loss = -activations.features[0, j].mean()` and an optimizer 
    `optimizer = torch.optim.Adam([img_var], lr=lr, weight_decay=1e-6)` that optimizes the pixel values. Optimizers by default minimize losses so instead of telling the optimizer to maximize the loss we simply multiply the mean activation by -1. Reset the gradients with `optimizer.zero_grad()`, calculate the gradients of the pixel values with `loss.backward()`, and change the pixel values with optimizer.step().